In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from symmetries.analysis.lie_symmetry_analysis import point_symmetries
from symmetries.utils.symbolic import sym_det_eqn

from symmetries.objects.general_form import GeneralForm
import sympy as sp
from sympy import Derivative as D

# One equation of Hall MHD 

In [4]:
x, y, z, t = sp.symbols('x y z t')

phi = sp.Function('phi')(x, y, z, t)
psi = sp.Function('psi')(x, y, z, t)
b = sp.Function('b')(x, y, z, t)
w = sp.Function('w')(x, y, z, t)

sigma, gamma = sp.symbols('sigma gamma')

list_indep = [x, y, z, t]
list_dep = [phi, psi, b, w]
list_cte = [sigma, gamma]

F =  D(b, t) + D(b, x)*D(phi, y) - D(b, y)*D(phi, x) + sigma*(D(psi, x)*(D(psi, x, x, y) + D(psi, y, y, y))-
        D(psi, y)*(D(psi, x, x, x) + D(psi, y, y, x))) + D(psi, x)*D(w, y) - D(psi, y)*D(w, x) - gamma*(D(b,x,x) + D(b, y, y))

F_rules_array = {D(psi, x, x, y): 
        - (sigma*D(psi, x))**(-1)*(D(b, t) + D(b, x)*D(phi, y) - D(b, y)*D(phi, x) 
        + D(psi, x)*D(w, y) - D(psi, y)*D(w, x) 
        - gamma*(D(b,x,x) + D(b, y, y)))
        +(D(psi, x))**(-1)*D(psi, y)*(D(psi, x, x, x) + D(psi, y, y, x))
        - D(psi, y, y, y)}

order = 3
F.expand()

-gamma*Derivative(b(x, y, z, t), (x, 2)) - gamma*Derivative(b(x, y, z, t), (y, 2)) + sigma*Derivative(psi(x, y, z, t), x)*Derivative(psi(x, y, z, t), (y, 3)) + sigma*Derivative(psi(x, y, z, t), x)*Derivative(psi(x, y, z, t), (x, 2), y) - sigma*Derivative(psi(x, y, z, t), (x, 3))*Derivative(psi(x, y, z, t), y) - sigma*Derivative(psi(x, y, z, t), y)*Derivative(psi(x, y, z, t), (y, 2), x) + Derivative(b(x, y, z, t), t) + Derivative(b(x, y, z, t), x)*Derivative(phi(x, y, z, t), y) - Derivative(b(x, y, z, t), y)*Derivative(phi(x, y, z, t), x) + Derivative(psi(x, y, z, t), x)*Derivative(w(x, y, z, t), y) - Derivative(psi(x, y, z, t), y)*Derivative(w(x, y, z, t), x)

# Heat Equation

In [3]:
x, t, k = sp.symbols('x t k')
u = sp.Function('u')(x, t)
independent_variables = [x, t]
dependent_variables = [u]
constants = [k]
F =  D(u,t) - k*D(u,x,x)
F_rules_array = {D(u,x,x):1/k*D(u,t)}
order = 2
F.expand()

-k*Derivative(u(x, t), (x, 2)) + Derivative(u(x, t), t)

In [4]:
model, system_of_equations = point_symmetries(F, order, F_rules_array, independent_variables, dependent_variables, constants)

In [5]:
general_form = GeneralForm(model, system_of_equations)

In [6]:
general_form.print_determining_equations()

Matrix([
[1,                                 Eq(xi^(t)_u_u, 0)],
[2,                                 Eq(xi^(x)_u_u, 0)],
[3,                                   Eq(xi^(t)_u, 0)],
[4,            Eq(-eta^(u)_u_u*k + 2*k*xi^(x)_x_u, 0)],
[5,                Eq(2*k*xi^(t)_x_u + 2*xi^(x)_u, 0)],
[6,                                   Eq(xi^(t)_x, 0)],
[7,       Eq(k*xi^(t)_x_x - xi^(t)_t + 2*xi^(x)_x, 0)],
[8, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[9,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])

In [8]:
general_form.simplify_iteratively()

deleting xit u(x, t)
deleting xit x
found x in xit eq 4
found u(x, t) in xit eq 4
found x in xit eq 6
found derivative of u(x, t) in xit eq 0
deleting xix u(x, t)
found u(x, t) in xix eq 3
found u(x, t) in xix eq 3
found derivative of u(x, t) in xix eq 1


In [9]:
general_form.print_general_form()

already deleted: {'xit': [u(x, t), x], 'xix': [u(x, t)]}
general form:


Matrix([
[eta(x, t, u(x, t))**u],
[             xi(t)**t],
[          xi(x, t)**x]])

In [10]:
general_form.print_determining_equations()

Matrix([
[1,                                   Eq(xi^(t)_u, 0)],
[2,                                Eq(eta^(u)_u_u, 0)],
[3,                                   Eq(xi^(x)_u, 0)],
[4,                                   Eq(xi^(t)_x, 0)],
[5,                     Eq(-xi^(t)_t + 2*xi^(x)_x, 0)],
[6, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[7,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])

# Blasius Equation

In [11]:
x = sp.symbols('x')
y = sp.Function('y')(x)
independent_variables = [x]
dependent_variables = [y]
constants = []
F =  D(y,x,x,x) + y*D(y,x,x)
F_rules_array = {D(y,x,x,x):-y*D(y,x,x)}
order = 3

In [12]:
F.expand()

y(x)*Derivative(y(x), (x, 2)) + Derivative(y(x), (x, 3))

In [13]:
model, system_of_equations = point_symmetries(F, order, F_rules_array, independent_variables, dependent_variables, constants)

In [14]:
general_form = GeneralForm(model, system_of_equations)

In [15]:
general_form.simplify_iteratively()
general_form.print_determining_equations()

deleting xix y(x)
found y(x) in xix eq 0
found y(x) in xix eq 0
found y(x) in xix eq 0
found y(x) in xix eq 4
found y(x) in xix eq 4
found y(x) in xix eq 4
found y(x) in xix eq 5
found y(x) in xix eq 5
found y(x) in xix eq 5
found y(x) in xix eq 5
found derivative of y(x) in xix eq 2
found derivative of y(x) in xix eq 3


Matrix([
[1,                                                         Eq(eta^(y)_y_y_y, 0)],
[2,                                                              Eq(xi^(x)_y, 0)],
[3,                                                Eq(3*eta^(y)_y_y + xi^(x), 0)],
[4,                                    Eq(3*eta^(y)_x_y_y + eta^(y)_y_y*y(x), 0)],
[5,                Eq(eta^(y) + 3*eta^(y)_x_y + xi^(x)_x*y(x) - 3*xi^(x)_x_x, 0)],
[6, Eq(3*eta^(y)_x_x_y + 2*eta^(y)_x_y*y(x) - xi^(x)_x_x*y(x) - xi^(x)_x_x_x, 0)],
[7,                                      Eq(eta^(y)_x_x*y(x) + eta^(y)_x_x_x, 0)]])

# Wave Equation

In [16]:
x, t, k = sp.symbols('x t k')
u = sp.Function('u')(x, t)
list_indep = [x, t]
list_dep = [u]
list_var = list_indep + list_dep
list_cte = [k]
list_all = list_cte + list_indep + list_dep
F =  D(u,t) - k*D(u,x,x)
F_rules_array = {D(u,x,x):1/k*D(u,t)}
order = 2
F.expand()

-k*Derivative(u(x, t), (x, 2)) + Derivative(u(x, t), t)

In [17]:
model, system_of_equations = point_symmetries(F, order, F_rules_array, list_indep, list_dep, list_cte)

In [18]:
general_form = GeneralForm(model, system_of_equations)

In [19]:
general_form.print_determining_equations()


Matrix([
[1,                                 Eq(xi^(t)_u_u, 0)],
[2,                                 Eq(xi^(x)_u_u, 0)],
[3,                                   Eq(xi^(t)_u, 0)],
[4,            Eq(-eta^(u)_u_u*k + 2*k*xi^(x)_x_u, 0)],
[5,                Eq(2*k*xi^(t)_x_u + 2*xi^(x)_u, 0)],
[6,                                   Eq(xi^(t)_x, 0)],
[7,       Eq(k*xi^(t)_x_x - xi^(t)_t + 2*xi^(x)_x, 0)],
[8, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[9,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])

# Schrödinger Equation

In [20]:
x, t, k, i = sp.symbols('x t k i')
u = sp.Function('u')(x, t)
list_indep = [x, t]
list_dep = [u]
list_cte = [k, i]
F =  -D(u,x,x) +x**2*u-i*k*D(u,t)
F_rules_array = {D(u,x,x):x**2-i*k*D(u,t)}
order = 2
F.expand()

-i*k*Derivative(u(x, t), t) + x**2*u(x, t) - Derivative(u(x, t), (x, 2))

In [21]:
model, system_of_equations = point_symmetries(F, order, F_rules_array, list_indep, list_dep, list_cte)

In [22]:
general_form = GeneralForm(model, system_of_equations)

In [23]:
general_form.print_determining_equations()

Matrix([
[1,                                                                                         Eq(xi^(t)_u_u, 0)],
[2,                                                                                         Eq(xi^(x)_u_u, 0)],
[3,                                                                                           Eq(xi^(t)_u, 0)],
[4,                                                                     Eq(-2*i*k*xi^(x)_u + 2*xi^(t)_x_u, 0)],
[5,                                                                        Eq(-eta^(u)_u_u + 2*xi^(x)_x_u, 0)],
[6,                                                                                           Eq(xi^(t)_x, 0)],
[7,                           Eq(i*k*xi^(t)_t - 2*i*k*xi^(x)_x + x**2*xi^(t) + x**2*xi^(t)_u + xi^(t)_x_x, 0)],
[8,                         Eq(-2*eta^(u)_x_u + i*k*xi^(x)_t + x**2*xi^(x) + 3*x**2*xi^(x)_u + xi^(x)_x_x, 0)],
[9, Eq(eta^(u)*x**2 - eta^(u)_t*i*k - eta^(u)_u*x**2 - eta^(u)_x_x + 2*x**2*xi^(x)_x + 2*x*xi^(

# HNLS Equation

In [26]:
# Real
z, t, g1, g2, l, s = sp.symbols('z t gamma_1 gamma_2 lambda_0 sigma')
u = sp.Function('u')(z, t)
v = sp.Function('v')(z, t)
list_indep = [z, t]
list_dep = [u, v]
list_cte = [g1, g2, l, s]
F =  D(u,t) + D(v,z,z) + l*(v*u**2+v**3)+s*D(u,z,z,z)+\
    g1*D(u,z)*(u**2+v**2)+2*g2*u*(u*D(u,z)+v*D(v,z))
F_rules_array = {D(u,z,z,z): (-1/s)*(D(u,t) + D(v,z,z) + l*(v*u**2+v**3)+
    g1*D(u,z)*(u**2+v**2)+2*g2*u*(u*D(u,z)+v*D(v,z)))}
order = 3
F.expand()
point_symmetries(F, order, F_rules_array, list_indep, list_dep, list_cte)

(<symmetries.objects.system.System at 0x7fed996fc3a0>,
 <symmetries.objects.determining_equations.DeterminingEquations at 0x7fed9986bdc0>)

In [ ]:
# Imaginaria
z, t, g1, g2, l, s = sp.symbols('z t gamma_1 gamma_2 lambda_0 sigma')
u = sp.Function('u')(z, t)
v = sp.Function('v')(z, t)
list_indep = [z, t]
list_dep = [u, v]
list_cte = [g1, g2, l, s]
F =-D(v,t) + D(u,z,z) + l*(v**2*u+u**3)-s*D(v,z,z,z)-\
    g1*D(v,z)*(u**2+v**2)-2*g2*v*(u*D(u,z)+v*D(v,z))
F_rules_array = {D(v,z,z,z): 1/s*(-D(v,t) + D(u,z,z) + l*(v**2*u+u**3)-\
    g1*D(v,z)*(u**2+v**2)-2*g2*v*(u*D(u,z)+v*D(v,z)))}
order = 3
F.expand()
point_symmetries(F, order, F_rules_array, list_indep, list_dep, list_cte)